In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    'jinaai/jina-reranker-v2-base-multilingual',
    torch_dtype="auto",
    trust_remote_code=True,
)

model.to('cuda') # or 'cpu' if no GPU is available
model.eval()


In [ ]:
# Example query and documents
query = "Comment fonctionnent les poumons ?"
documents = [
    "Le sang emporte le gaz carbonique vers les poumons qui le rejettent.",
    "La combustion de la poudre ne va pas forcément produire une explosion. La poudre simplement répandue à terre va se comporter comme une mèche, elle brûle progressivement sans exploser.",
    "Comment fonctionnent les feux d'artifice ? La fabrication et l'utilisation des feux d'artifice relèvent de la pyrotechnie qui regroupe essentiellement les techniques qui consistent à créer des explosions à partir d'une combustion.",
    "On peut aussi rajouter des petites granules d'aluminium pour obtenir une pluie d'étincelles. Enfin, chaque bombe peut contenir d'autres bombes qui vont s'éparpiller dans le ciel, créant ainsi des explosions secondaires avec différents effets.",
    "C'est primordial de vérifier parce que nous respirons cette air et cette air entre dans nos poumons et nous risquons une irradiation interne.",
    "Les poumons sont composés de conduits, les bronches, qui comme les branches d'un arbre se ramifient en voies respiratoires plus petites, les bronchioles.",
    "Les poumons sont composés de conduits, les bronches qui comme les branches d'un arbre se ramifient en voies respiratoires plus petites, les bronchioles.",
    "Le gaz occupe plus de place que le matériau initial et la température augmente encore son volume, ce qui suffit à provoquer l'explosion. La combustion qui va déclencher l'explosion est une réaction chimique entre un combustible, comme du bois, et un comburant, très souvent l'oxygène de l'air. Et dans le cas d'une explosion, on ne peut pas trop compter sur l'oxygène de l'air, car sa concentration est insuffisante pour produire une combustion suffisamment violente pour produire une explosion.",
    "L'air pénètre donc dans les poumons jusqu'au fond des alvéoles. C'est là que se produisent les échanges gazeux entre l'air et le sang. C'est-à-dire que le dioxygène quitte l'air pour passer dans le sang et le dioxyde de carbone s'échappe du sang pour être rejeté dans l'air.",
    "Sous-titrage ST'501",
    "Ces explosifs peuvent servir à propulser les engins et à les faire exploser.",
    "Lorsque la poudre explose, les granules sont projetées dans toutes les directions, mais en gardant leur disposition initiale. Et la chaleur de l'explosion initie la réaction de combustion et donc l'émission de lumière.",
    "Les explosifs utilisés lors des feux d'artifice sont du même type que la poudre à canon.",
    "Et pour déclencher la combustion, il suffit d'apporter une relativement faible énergie d'activation sous la forme d'une flambe ou d'un courant électrique.",
    "Pour obtenir l'effet souhaité, il faut d'abord que la bombe atteigne l'altitude visée. Pour cela, deux façons de procéder. La première consiste à prévoir dans la bombe un étage de propulsion, exactement comme pour une fusée.",
    "Il est obtenu à l'aide de ce que l'on appelle une bombe de feu d'artifice.",
    "Ces granules sont disposées autour de la poudre en respectant le dessin que l'on veut obtenir au final.",
    "La solution consiste alors à utiliser un mélange contenant à la fois le combustible et le comburant. C'est ce qu'on appelle un explosif.",
    "Par exemple, un composé à base de strontium émet de la lumière rouge, un composé à base de cuivre émet de la lumière bleue.",
    "Prenons l'exemple de l'effet le plus répandu, qu'on appelle la pivoine, et qui dessine une fleur sphérique."
]
# construct sentence pairs
sentence_pairs = [[query, doc] for doc in documents]

scores = model.compute_score(sentence_pairs, max_length=1024)


sorted_scores = sorted(zip(scores, documents), reverse=True)

for score, doc in sorted_scores:
    print(doc)
    print(score)
    print("============")


## Test SIReranker

In [1]:
import sys
sys.path.insert(0, '../app')
sys.path.insert(0, '../')
from app.processing.text.SIReranker import SIReranker


reranker = SIReranker()

/home/erwan/Desktop/clients/ScienceInfuse/server/notebooks/../app/schemas.py:182: GenericBeforeBaseModelWarning: Classes should inherit from `BaseModel` before generic classes (e.g. `typing.Generic[T]`) for pydantic generics to work properly.
  class ChunkWithScore(Generic[ChunkType], BaseDocumentChunk):


In [2]:

import sys
import os
sys.path.insert(0, '../')
sys.path.insert(0, '../app')


from pydantic import BaseModel
import weaviate
from app.search.search_chunks import search_all_chunks

query = "dinosaures"
client = weaviate.connect_to_local()
chunks = search_all_chunks(client=client, query=query).chunks
client.close()

reranker.sort_docment_chunks(document_chunks=chunks, query=query)

ALL PROPS ['uuid', 'document_id', 'public_path', 'original_path', 's3_object_name', 'user_approved', 'user_disapproved', 'media_name', 'document', 'text', 'title', 'media_type', 'meta_start', 'meta_end', 'meta_s3_object_name', 'meta_page_number', 'meta_bbox', 'meta_s3_object_name', 'meta_page_number', 'meta_bbox', 'meta_question', 'meta_answer', 'meta_url', 'meta_description', 'meta_title', 'meta_type', 'meta_url']
QUERY dinosaures


[ChunkWithScore(uuid='db86e058-bfbc-41d9-8b0f-8dfb0e77a8b0', document=Document(uuid='32782090-acb0-44b6-9604-b6239a561aa2', document_id='70cc4c0c-7d44-4ac7-a241-a0fb8820e380', public_path='', original_path='/home/erwan/Desktop/clients/ScienceInfuse/server/ftp-data/2015 - Dinosaures.pdf', s3_object_name='pdf/70cc4c0c-7d44-4ac7-a241-a0fb8820e380.pdf', user_approved=None, user_disapproved=None, media_name='2015 - Dinosaures'), text='Le terme dinosaure est forgé à partir de deux mots du grec ancien : δεινός (deinόs, terrible) et σαῦρος (saûros, lézard). Ce nom est toutefois quelque peu trompeur car les dinosaures ne sont pas des lézards ; ils représentent un clade distinct de reptiles apparus au cours de la période Trias il y a 230 millions d’années. Rappelons qu’un clade est un groupe d’organismes vivants ou ayant vécu comprenant un organisme particulier et la totalité de ses descendants.', title='II.2 Un mot sur les échelles de temps géologiques>II.3 Généralités sur les dinosaures', medi